#### Import project_lib to be able to save the final data to csv

In [1]:
# The code was removed by Watson Studio for sharing.

### import packages

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
!pip install folium
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors


/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes


### Create a beautiful soup object

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html.parser')

### Get London neighborhood data into a list

In [4]:
table_contents = []
table = soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span == None:
        pass
    elif row.span.text.split("(")[0] =='Y':
        pass
    else: 
        cell['Borough'] = row.span.text.split("/")[2].split("(")[1].strip(')')
        cell['Latitude'] = float(row.span.text.split("/")[2].split("(")[0][:8])
        cell['Longitude'] = row.span.text.split("/")[2].split(";")[1].split(' ')[1][:-1]
        table_contents.append(cell)


### Convert data into a dataframe and add one more row of City of London as it was in a separate table on Wikipedia

In [5]:
df = pd.DataFrame(table_contents)
new_row = {'Borough': 'City of London', 'Latitude': 51.5155, 'Longitude': -0.0922 }
df_london = df.append(new_row, ignore_index = True)

### Check data

In [6]:

df_london

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198
5,Camden,51.5290,-0.1255
6,Croydon,51.3714,-0.0977
7,Ealing,51.5130,-0.3089
8,Enfield,51.6538,-0.0799
9,Greenwich,51.4892,0.0648


In [7]:
df_london.shape

(33, 3)

### Create map of London using the latitude and longitude values

In [8]:
map_london = folium.Map(location=[51.5155,-0.0922], zoom_start=11)
for lat, lng, label in zip(df_london['Latitude'], df_london['Longitude'], df_london['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london


### Define foursquare credential

In [9]:
CLIENT_ID = 'PTBQE2DOOJIFNTKH5YLSOCAWN1AS3EWCINOX0BUH30FH5E0V' # Foursquare ID
CLIENT_SECRET = 'IRVTNCASERYLWAVUBRMEO1AAYQAIG4VK13U5FU2CWZVSZLSZ' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: PTBQE2DOOJIFNTKH5YLSOCAWN1AS3EWCINOX0BUH30FH5E0V
CLIENT_SECRET:IRVTNCASERYLWAVUBRMEO1AAYQAIG4VK13U5FU2CWZVSZLSZ


## 

### Create a function to get venues

In [10]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Get venues

In [11]:
Lon_venues = getNearbyVenues(names=df_london['Borough'],
                                   latitudes=df_london['Latitude'],
                                   longitudes=df_london['Longitude'])


Barking and Dagenham
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich
Hackney
Hammersmith and Fulham
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster
City of London


In [12]:
Lon_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,51.5607,0.1557,Central Park,51.559560,0.161981,Park
1,Barking and Dagenham,51.5607,0.1557,Morrisons,51.559774,0.148752,Supermarket
2,Barking and Dagenham,51.5607,0.1557,Crowlands Heath Golf Course,51.562457,0.155818,Golf Course
3,Barking and Dagenham,51.5607,0.1557,Robert Clack Leisure Centre,51.560808,0.152704,Martial Arts School
4,Barking and Dagenham,51.5607,0.1557,Beacontree Heath Leisure Centre,51.560997,0.148932,Gym / Fitness Center


### Replace all venue category with Restaurant and cafe in it with one single category "Restaurant"

In [13]:

Lon_venues['Venue Category'] = np.where(Lon_venues['Venue Category'].str.contains('Restaurant'), 'Restaurant', Lon_venues['Venue Category'])

Lon_venues['Venue Category'] = np.where(Lon_venues['Venue Category'].str.contains('Café'), 'Restaurant', Lon_venues['Venue Category'])


In [14]:
Lon_venues.head(20)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,51.5607,0.1557,Central Park,51.559560,0.161981,Park
1,Barking and Dagenham,51.5607,0.1557,Morrisons,51.559774,0.148752,Supermarket
2,Barking and Dagenham,51.5607,0.1557,Crowlands Heath Golf Course,51.562457,0.155818,Golf Course
3,Barking and Dagenham,51.5607,0.1557,Robert Clack Leisure Centre,51.560808,0.152704,Martial Arts School
4,Barking and Dagenham,51.5607,0.1557,Beacontree Heath Leisure Centre,51.560997,0.148932,Gym / Fitness Center
5,Barking and Dagenham,51.5607,0.1557,Becontree Heath Bus Station,51.561065,0.150998,Bus Station
6,Barking and Dagenham,51.5607,0.1557,Dagenham Swimming Pool,51.560946,0.150054,Pool
7,Barnet,51.6252,-0.1517,The Atrium,51.624726,-0.151933,Restaurant
8,Barnet,51.6252,-0.1517,Beaconsfield Road (BF),51.622827,-0.151466,Bus Stop
9,Barnet,51.6252,-0.1517,Oakleigh Cafe,51.623412,-0.154899,Restaurant


### Count number of venues in each neighborhood

In [15]:
Lon_venuecount=Lon_venues.groupby('Neighborhood').count().reset_index()
Lon_venuecount

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Barking and Dagenham,7,7,7,7,7,7
1,Barnet,4,4,4,4,4,4
2,Bexley,28,28,28,28,28,28
3,Brent,77,77,77,77,77,77
4,Bromley,37,37,37,37,37,37
5,Camden,80,80,80,80,80,80
6,City of London,100,100,100,100,100,100
7,Croydon,38,38,38,38,38,38
8,Ealing,74,74,74,74,74,74
9,Enfield,52,52,52,52,52,52


### Check how many categories are there. 

In [16]:
print('There are {} uniques categories.'.format(len(Lon_venues['Venue Category'].unique())))

There are 173 uniques categories.


### Analyze each neighborhood

In [17]:
Lon_onehot = pd.get_dummies(Lon_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Lon_onehot['Neighborhood'] = Lon_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [Lon_onehot.columns[-1]] + list(Lon_onehot.columns[:-1])
Lon_onehot = Lon_onehot[fixed_columns]

Lon_onehot.head()

,Neighborhood,Airport Lounge,Airport Service,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,BBQ Joint,Baby Store,Bakery,...,Toy / Game Store,Train Station,Tunnel,Used Bookstore,Video Game Store,Warehouse Store,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Barking and Dagenham,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Group rows by neighborhood and take the mean of each venue category

In [18]:
Lon_grouped = Lon_onehot.groupby('Neighborhood').mean().reset_index()
Lon_grouped

,Neighborhood,Airport Lounge,Airport Service,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,BBQ Joint,Baby Store,Bakery,...,Toy / Game Store,Train Station,Tunnel,Used Bookstore,Video Game Store,Warehouse Store,Wine Bar,Wine Shop,Winery,Yoga Studio
0,Barking and Dagenham,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Barnet,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Bexley,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.035714,...,0.000000,0.000000,0.000000,0.0000,0.035714,0.035714,0.000000,0.000000,0.00,0.000000
3,Brent,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.038961,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Bromley,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.027027,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
5,Camden,0.000000,0.000000,0.000000,0.012500,0.000000,0.00,0.00,0.00,0.025000,...,0.000000,0.025000,0.000000,0.0125,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
6,City of London,0.000000,0.000000,0.000000,0.030000,0.000000,0.00,0.00,0.00,0.020000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.030000,0.000000,0.00,0.020000
7,Croydon,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
8,Ealing,0.000000,0.000000,0.000000,0.013514,0.000000,0.00,0.00,0.00,0.040541,...,0.000000,0.000000,0.000000,0.0000,0.013514,0.013514,0.013514,0.000000,0.00,0.000000
9,Enfield,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.019231,...,0.000000,0.019231,0.000000,0.0000,0.019231,0.000000,0.000000,0.000000,0.00,0.000000


### Create a dataframe with only Restaurant data

In [19]:
Lon_restaurant = pd.concat([Lon_grouped['Neighborhood'],Lon_grouped['Restaurant']], axis = 1)
Lon_restaurant

,Neighborhood,Restaurant
0,Barking and Dagenham,0.000000
1,Barnet,0.500000
2,Bexley,0.214286
3,Brent,0.168831
4,Bromley,0.135135
5,Camden,0.262500
6,City of London,0.300000
7,Croydon,0.394737
8,Ealing,0.297297
9,Enfield,0.173077


 ### Print each neighborhood along with the top 5 most common venues

In [20]:
num_top_venues = 5

for hood in Lon_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Lon_grouped[Lon_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham----
                  venue  freq
0                  Pool  0.14
1  Gym / Fitness Center  0.14
2           Bus Station  0.14
3   Martial Arts School  0.14
4           Supermarket  0.14


----Barnet----
                        venue  freq
0                  Restaurant  0.50
1  Construction & Landscaping  0.25
2                    Bus Stop  0.25
3              Airport Lounge  0.00
4                        Park  0.00


----Bexley----
            venue  freq
0      Restaurant  0.21
1  Clothing Store  0.14
2             Pub  0.11
3     Supermarket  0.07
4     Coffee Shop  0.07


----Brent----
            venue  freq
0      Restaurant  0.17
1     Coffee Shop  0.10
2           Hotel  0.09
3  Clothing Store  0.08
4             Bar  0.06


----Bromley----
            venue  freq
0  Clothing Store  0.16
1     Coffee Shop  0.14
2      Restaurant  0.14
3     Pizza Place  0.05
4             Bar  0.05


----Camden----
           venue  freq
0     Restaurant  0.26
1          H

### Create a function that sorts the venues in decending order 

In [21]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Create a new dataframe that display the top five most common venues in each neighborhood

In [22]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Lon_grouped['Neighborhood']

for ind in np.arange(Lon_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Lon_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,Pool,Gym / Fitness Center,Martial Arts School,Golf Course,Bus Station
1,Barnet,Restaurant,Construction & Landscaping,Bus Stop,Dog Run,Fish & Chips Shop
2,Bexley,Restaurant,Clothing Store,Pub,Pharmacy,Coffee Shop
3,Brent,Restaurant,Coffee Shop,Hotel,Clothing Store,Bar
4,Bromley,Clothing Store,Restaurant,Coffee Shop,Burger Joint,Gym / Fitness Center


## Clustering neighborhood

### Run K-means to cluster the neighnorhood into 5 clusters

In [23]:
# set number of clusters
kclusters = 5

Lon_grouped_clustering = Lon_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Lon_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([2, 4, 3, 3, 3, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 3, 3, 0, 1, 1],
      dtype=int32)

### Create a new dataframe to include the clusters

In [24]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Lon_merged = df_london.rename(columns={'Borough' : 'Neighborhood'})

# merge Lon_grouped with df_londonto add latitude/longitude for each neighborhood
Lon_merged = Lon_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Lon_merged.head() # check the last columns!

,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,51.5607,0.1557,2,Pool,Gym / Fitness Center,Martial Arts School,Golf Course,Bus Station
1,Barnet,51.6252,-0.1517,4,Restaurant,Construction & Landscaping,Bus Stop,Dog Run,Fish & Chips Shop
2,Bexley,51.4549,0.1505,3,Restaurant,Clothing Store,Pub,Pharmacy,Coffee Shop
3,Brent,51.5588,-0.2817,3,Restaurant,Coffee Shop,Hotel,Clothing Store,Bar
4,Bromley,51.4039,0.0198,3,Clothing Store,Restaurant,Coffee Shop,Burger Joint,Gym / Fitness Center


### Show clustering on the map

In [25]:
# create map
map_clusters = folium.Map(location=[51.5155,-0.0922], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Lon_merged['Latitude'], Lon_merged['Longitude'], Lon_merged['Neighborhood'], Lon_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.9).add_to(map_clusters)
       
map_clusters

## Examining clusters

### Clusters from 1 to 5 showed below

In [26]:
Lon_merged.loc[Lon_merged['Cluster Labels'] == 0, Lon_merged.columns[[0] + list(range(3, Lon_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
16,Hounslow,0,Bed & Breakfast,Metro Station,Restaurant,Park,Discount Store


In [27]:
Lon_merged.loc[Lon_merged['Cluster Labels'] == 1, Lon_merged.columns[[0] + list(range(3, Lon_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Camden,1,Restaurant,Coffee Shop,Hotel,Pub,Deli / Bodega
6,Croydon,1,Restaurant,Pub,Coffee Shop,Park,Clothing Store
7,Ealing,1,Restaurant,Coffee Shop,Clothing Store,Park,Pub
11,Hammersmith and Fulham,1,Restaurant,Pub,Coffee Shop,Clothing Store,Gastropub
12,Haringey,1,Restaurant,Movie Theater,Pub,Supermarket,Park
13,Harrow,1,Restaurant,Coffee Shop,Indie Movie Theater,Grocery Store,Supermarket
17,Islington,1,Restaurant,Pub,Park,Cocktail Bar,Music Venue
18,Kensington and Chelsea,1,Restaurant,Juice Bar,Clothing Store,Gym / Fitness Center,Burger Joint
19,Kingston upon Thames,1,Restaurant,Coffee Shop,Clothing Store,Pub,Burger Joint
20,Lambeth,1,Restaurant,Market,Coffee Shop,Pub,Pizza Place


In [28]:
Lon_merged.loc[Lon_merged['Cluster Labels'] == 2, Lon_merged.columns[[0] + list(range(3, Lon_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Barking and Dagenham,2,Pool,Gym / Fitness Center,Martial Arts School,Golf Course,Bus Station


In [29]:
Lon_merged.loc[Lon_merged['Cluster Labels'] == 3, Lon_merged.columns[[0] + list(range(3, Lon_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,Bexley,3,Restaurant,Clothing Store,Pub,Pharmacy,Coffee Shop
3,Brent,3,Restaurant,Coffee Shop,Hotel,Clothing Store,Bar
4,Bromley,3,Clothing Store,Restaurant,Coffee Shop,Burger Joint,Gym / Fitness Center
8,Enfield,3,Restaurant,Coffee Shop,Clothing Store,Supermarket,Pub
9,Greenwich,3,Restaurant,Pub,Supermarket,Coffee Shop,Clothing Store
10,Hackney,3,Restaurant,Pub,Coffee Shop,Brewery,Bakery
14,Havering,3,Coffee Shop,Restaurant,Clothing Store,Shopping Mall,Hotel
15,Hillingdon,3,Restaurant,Coffee Shop,Clothing Store,Pub,Pharmacy
21,Lewisham,3,Restaurant,Supermarket,Grocery Store,Coffee Shop,Platform
23,Newham,3,Hotel,Sandwich Place,Airport Lounge,Light Rail Station,Airport Service


In [30]:
Lon_merged.loc[Lon_merged['Cluster Labels'] == 4, Lon_merged.columns[[0] + list(range(3, Lon_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Barnet,4,Restaurant,Construction & Landscaping,Bus Stop,Dog Run,Fish & Chips Shop


## Load airbnb listing data

In [31]:
# The code was removed by Watson Studio for sharing.

In [32]:
listing.head()

,neighbourhood_cleansed,accommodates,price
0,Lambeth,4,110.0
1,Islington,2,40.0
2,Kensington and Chelsea,2,75.0
3,Westminster,6,307.0
4,Hammersmith and Fulham,2,150.0


### Calculate price per bed

In [33]:
listing['price_per_bed'] = listing['price'] / listing['accommodates']

In [34]:
listing.head()

,neighbourhood_cleansed,accommodates,price,price_per_bed
0,Lambeth,4,110.0,27.500000
1,Islington,2,40.0,20.000000
2,Kensington and Chelsea,2,75.0,37.500000
3,Westminster,6,307.0,51.166667
4,Hammersmith and Fulham,2,150.0,75.000000


In [35]:
listing = listing.groupby('neighbourhood_cleansed').mean().reset_index()
listing.rename(columns={"neighbourhood_cleansed":"Neighborhood"}, inplace=True)

In [36]:
listing.columns


Index(['Neighborhood', 'accommodates', 'price', 'price_per_bed'], dtype='object')

### Combine listing and venue files

In [37]:
London_merged = Lon_venuecount.merge(listing.set_index('Neighborhood'), on='Neighborhood')
London_merged = London_merged.merge(Lon_restaurant.set_index('Neighborhood'), on = 'Neighborhood')

In [38]:
London_merged.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,accommodates,price,price_per_bed,Restaurant
0,Barking and Dagenham,7,7,7,7,7,7,2.606516,63.922306,30.109057,0.000000
1,Barnet,4,4,4,4,4,4,2.939450,100.646483,48.427835,0.500000
2,Bexley,28,28,28,28,28,28,2.458484,57.339350,25.657597,0.214286
3,Brent,77,77,77,77,77,77,2.984449,97.952484,36.348829,0.168831
4,Bromley,37,37,37,37,37,37,2.701599,72.701599,29.612419,0.135135


### Drop unnecessary columns

In [39]:
London_merged.drop(columns=['Neighborhood Latitude' , 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue Category'], inplace=True)

London_merged.head()

,Neighborhood,Venue,accommodates,price,price_per_bed,Restaurant
0,Barking and Dagenham,7,2.606516,63.922306,30.109057,0.000000
1,Barnet,4,2.939450,100.646483,48.427835,0.500000
2,Bexley,28,2.458484,57.339350,25.657597,0.214286
3,Brent,77,2.984449,97.952484,36.348829,0.168831
4,Bromley,37,2.701599,72.701599,29.612419,0.135135


### Sort data by price per bed 

In [40]:
London_sorted = London_merged.sort_values(['price_per_bed'], ascending=False)
London_sorted.head(10)

,Neighborhood,Venue,accommodates,price,price_per_bed,Restaurant
32,Westminster,73,3.390317,186.926001,58.858879,0.205479
19,Kensington and Chelsea,54,3.420190,181.939429,53.871180,0.351852
6,City of London,100,3.066986,157.722488,53.770455,0.300000
1,Barnet,4,2.939450,100.646483,48.427835,0.500000
18,Islington,48,2.997682,135.591331,47.456859,0.229167
5,Camden,80,3.182067,133.259035,46.974380,0.262500
23,Merton,19,2.990909,107.325620,41.267208,0.263158
12,Hammersmith and Fulham,79,3.243267,122.923009,39.996121,0.405063
27,Southwark,100,2.971787,105.307933,38.845497,0.300000
8,Ealing,74,2.778046,96.381182,38.694530,0.297297


### Sort data by number of venues

In [41]:
London_merged.sort_values(['Venue'], ascending=False)


,Neighborhood,Venue,accommodates,price,price_per_bed,Restaurant
27,Southwark,100,2.971787,105.307933,38.845497,0.300000
21,Lambeth,100,2.892414,90.824427,32.707399,0.340000
6,City of London,100,3.066986,157.722488,53.770455,0.300000
20,Kingston upon Thames,100,2.808989,93.250936,38.038280,0.320000
5,Camden,80,3.182067,133.259035,46.974380,0.262500
12,Hammersmith and Fulham,79,3.243267,122.923009,39.996121,0.405063
3,Brent,77,2.984449,97.952484,36.348829,0.168831
8,Ealing,74,2.778046,96.381182,38.694530,0.297297
32,Westminster,73,3.390317,186.926001,58.858879,0.205479
11,Hackney,62,2.852403,89.694225,33.826345,0.177419


### Sort data by restaurant data

In [42]:
London_merged.sort_values(['Restaurant'], ascending=False)

,Neighborhood,Venue,accommodates,price,price_per_bed,Restaurant
1,Barnet,4,2.939450,100.646483,48.427835,0.500000
12,Hammersmith and Fulham,79,3.243267,122.923009,39.996121,0.405063
7,Croydon,38,2.410909,56.794545,25.971782,0.394737
13,Haringey,18,2.869278,86.235960,34.608258,0.388889
19,Kensington and Chelsea,54,3.420190,181.939429,53.871180,0.351852
21,Lambeth,100,2.892414,90.824427,32.707399,0.340000
14,Harrow,6,2.452055,71.294521,31.383029,0.333333
20,Kingston upon Thames,100,2.808989,93.250936,38.038280,0.320000
27,Southwark,100,2.971787,105.307933,38.845497,0.300000
6,City of London,100,3.066986,157.722488,53.770455,0.300000


### Save data to project to be used for dashboard

In [44]:

project.save_data('london_merged.csv', London_merged.to_csv(index=False))